In [2]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchvision.transforms import ToTensor, Grayscale

# Load VGG19 model
vgg19 = models.vgg19(pretrained=True)

# Modify VGG19 to accept 1-channel input
vgg19.features[0] = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Freeze VGG19 parameters
for param in vgg19.parameters():
    param.requires_grad = False

    
    

#Resnet Architecture

def conv3x3(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,padding=1,bias=False)

def conv1x1(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=1,stride=stride,bias=False)

# Load ResNet model
# Define the ResNet block and layer functions
# ...

# Define the ResNet model
import torch
import torch.nn as nn

# Define the basic block for ResNet
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Usage example:
resnet = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=2)


# Combine the models
class CombinedModel(nn.Module):
    def __init__(self, vgg_model, resnet_model):
        super(CombinedModel, self).__init__()
        self.vgg = vgg_model
        self.resnet = resnet_model
        self.fc = nn.Linear(1000 + 2, 2)  # Assuming the last layer has 1000 units in VGG19

    def forward(self, x):
        # Convert RGB images to grayscale
        x = Grayscale()(x)
        if not isinstance(x, torch.Tensor):
            x = TF.to_tensor(x)
        vgg_features = self.vgg(x)
        resnet_features = self.resnet(x)
        features = torch.cat((vgg_features, resnet_features), dim=1)
        output = self.fc(features)
        return output

# Create the combined model
combined_model = CombinedModel(vgg19, resnet)

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Electronica Care/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|███████████████████████████████████████████████████████████████████████████████| 548M/548M [07:00<00:00, 1.37MB/s]


In [4]:
# Create the combined model
combined_model = CombinedModel(vgg19, resnet)

combined_model.load_state_dict(torch.load("300epoch_VGG.pth",map_location=torch.device('cpu')))

<All keys matched successfully>